In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install -q pydrive

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [5]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [6]:
!pip install transformers

     |████████████████████████████████| 2.3MB 4.2MB/s 
     |████████████████████████████████| 901kB 42.5MB/s 
     |████████████████████████████████| 3.3MB 50.2MB/s 


In [7]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 4.3MB/s 
     |████████████████████████████████| 225kB 44.3MB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
     |████████████████████████████████| 1.8MB 42.9MB/s 
     |████████████████████████████████| 122kB 58.7MB/s 
     |████████████████████████████████| 8.2MB 29.2MB/s 
     |████████████████████████████████| 1.2MB 37.2MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 245kB 42.5MB/s 
     |████████████████████████████████| 112kB 43.4MB/s 
     |████████████████████████████████| 102kB 11.8MB/s 
     |████████████████████████████████| 174kB 53.9MB/s 
     |████████████████████████████████| 133kB 54.4MB/s 
     |████████████████████████████████| 112kB 46.7MB/s 
     |████████████████████████████████| 4.2MB 24.3MB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
     |████████████████████████████████| 122kB 45.0MB/s

In [8]:
project_path='/content/drive/My Drive/NLP/'

In [19]:
import numpy as np
import pandas as pd

def create_dataset():
  fake = pd.read_csv(project_path+'fake_and_real_news/Fake.csv')
  real = pd.read_csv(project_path+'fake_and_real_news/True.csv')
  fake['label'] = np.zeros((len(fake['text'])))
  real['label'] = np.ones((len(real['text'])))
  #df = pd.DataFrame({'text':[], 'label':[]}, columns=['text', 'label']);
  combined = pd.concat([fake, real])
  return combined

dataset = create_dataset()
dataset = dataset.sample(frac=1).reset_index(drop=True)

#dataset = pd.read_csv(project_path+'fake_or_real_news.csv')
print(dataset.shape)

texts=[]
texts=dataset['text']#####################################
label=dataset['label']

from sklearn.preprocessing import LabelEncoder
labelEncoder=LabelEncoder()
encoded_label=labelEncoder.fit_transform(label)
y=np.reshape(encoded_label,(-1,1))
print(y.shape)


#X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

training_size=int(0.8*dataset.shape[0])
print(dataset.shape[0],training_size)
data_train=dataset[:training_size]['text']
y_train_val=y[:training_size]
data_test=dataset[training_size:]['text']
y_test=y[training_size:]


(44898, 5)
(44898, 1)
44898 35918


In [20]:
dataset

,title,text,subject,date,label
0,Criminal Complaint Shows Tie Between Alex Jon...,Conspiracy theorist Alex Jones isn t known for...,News,"December 13, 2016",0.0
1,GUT WRENCHING: OBAMA Meets With Rappers To Dis...,Is there any question what Obama s priorities ...,left-news,"Apr 15, 2016",0.0
2,"Is Hillary’s Meltdown Real, or a Staged Exit?",21st Century Wire says As shocking and controv...,Middle-east,"September 14, 2016",0.0
3,"Trump Issues Warning To “200,000 man army” Tha...",Before you blame Trump for not being compassio...,politics,"Oct 1, 2015",0.0
4,Fox News Host Makes Herself A Total Laughings...,Fox News host Greta Van Susteren saw an opport...,News,"June 2, 2016",0.0
...,...,...,...,...,...
44893,ANTI-HILLARY POSTERS POP UP All Over Hollywood...,The first time I heard Negan ask for 50 perc...,politics,"Oct 22, 2016",0.0
44894,Russia: Moscow does not want to escalate situa...,MOSCOW (Reuters) - Moscow does not want to esc...,politicsNews,"September 12, 2017",1.0
44895,Panic on London's Oxford Street after reports ...,LONDON (Reuters) - Panic erupted among Christm...,worldnews,"November 24, 2017",1.0
44896,BREAKING: Did Hillary’s Unsecured CLASSIFIED E...,Will the State Department ever tell the truth ...,politics,"Aug 8, 2016",0.0


In [23]:
sentences = data_train
print(sentences)
labels = y_train_val[:,0]
#labels = y_train
print(labels)
print(1 in labels)

0        Conspiracy theorist Alex Jones isn t known for...
1        Is there any question what Obama s priorities ...
2        21st Century Wire says As shocking and controv...
3        Before you blame Trump for not being compassio...
4        Fox News host Greta Van Susteren saw an opport...
                               ...                        
35913    CHARLESTON, S.C. (Reuters) - The Republican pr...
35914    BERLIN (Reuters) - Germany will experience tra...
35915    As father and son comedy writers and liberal a...
35916    Most political campaigns prefer to spend the b...
35917    While Republican presidential frontrunner Dona...
Name: text, Length: 35918, dtype: object
[0 0 0 ... 0 0 0]
True


In [24]:
print('Max sentence length: ', max([len(sen.split(' ')) for sen in sentences]))

Max sentence length:  8436


In [25]:
print(sentences[:1])

0    Conspiracy theorist Alex Jones isn t known for...
Name: text, dtype: object


In [27]:
TRAIN_SPLIT = 0.8

nb_train_samples = int(TRAIN_SPLIT * sentences.shape[0])

x_train = np.array(sentences[:nb_train_samples])
y_train = np.array(labels[:nb_train_samples])
x_val = np.array(sentences[nb_train_samples:])
y_val = np.array(labels[nb_train_samples:])


In [28]:
import pandas as pd
ll=[]
for i in range(len(x_train)):
  ll.append([x_train[i],y_train[i]])

train_df = pd.DataFrame(ll,columns=['text','labels'])


ll=[]
for i in range(len(x_val)):
  ll.append([x_val[i],y_val[i]])

val_df = pd.DataFrame(ll,columns=['text','labels'])

In [29]:
sentences_test = np.array(data_test)
print(sentences_test[0])
labels_test = y_test[:, 0]
#labels = y_train
print(labels_test)

HONG KONG (Reuters) - Macau s mostly pro-democracy opposition made slight gains in legislative elections, as critics saw signs of public dissatisfaction with the pro-Beijing government after a deadly typhoon battered the Asian gambling hub a month ago.  Results of Sunday s vote announced on Monday showed the opposition camp won five seats in the 33-seat legislative assembly, one more than at the last election four years ago, according to local media reports. Macau s political landscape has long been dominated by pro-establishment, pro-Beijing and casino industry interests, and critics viewed any improvement by the opposition as an achievement.  Only 14 of the legislative seats were directly elected, a far lower proportion than in neighboring Hong Kong, also a China-ruled former colony, that has pushed much harder and wrested greater democratic freedoms than Macau, which returned from Portuguese to Chinese rule in 1999. Twelve other seats are indirectly elected in smaller constituencies

In [30]:
ll=[]
for i in range(len(sentences_test)):
  ll.append([sentences_test[i],labels_test[i]])

test_df = pd.DataFrame(ll,columns=['text','labels'])

In [31]:
print('Training sample: ' + str(len(train_df['text'])))
print('Validation sample: ' + str(len(val_df['text'])))
print('Testing sample: ' + str(len(test_df['text'])))

Training sample: 28734
Validation sample: 7184
Testing sample: 8980


In [33]:
model_type = "roberta"
model_name = "roberta-base"

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": "/content/drive/My Drive/NLP/Transformer/output/"+model_type,
    "best_model_dir": "/content/drive/My Drive/NLP/Transformer/output/"+model_type+"/best_model",
    "use_early_stopping": False,
    "early_stopping_delta": 0.0,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize" : True,
    "early_stopping_patience" : 2,
    "evaluate_during_training": True,
    "max_seq_length": 512,
    "num_train_epochs": 10,
    "evaluate_during_training_steps": 64,
    "wandb_project": "Fake News FR_1 RoBerta",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "train_batch_size": 64,
    "eval_batch_size": 64,
    "evaluate_during_training_verbose" : True
}

In [40]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
#import pandas as pd

train_args["max_seq_length"] = 300
train_args["train_batch_size"] = 32
train_args["gradient_accumulation_steps"] = 1
train_args["evaluate_during_training"] = True
train_args["num_train_epochs"] = 10
train_args["use_early_stopping"] = False


# Create a ClassificationModel
model = ClassificationModel(model_type, model_name, args=train_args)

# Train the model
model.train_model(train_df,eval_df = val_df)
#torch.save(model.state_dict(), "/content/drive/My Drive/NLP/roberta_embed.pt")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.

_runtime,12
_timestamp,1621011363
_step,2


_runtime,▁▅█
_timestamp,▁▅█
_step,▁▅█


(8980,
 {'auprc': [0.9942868570603907,
   0.9999757523648349,
   0.9999951127202196,
   0.9999990798526949,
   1.0,
   1.0,
   0.9999931495116682,
   0.9999911681620832,
   0.9999931457873886,
   1.0,
   0.9999999999999999,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.9999954277487589,
   0.9999862027945184,
   0.9999865288977041,
   0.9999996658275946,
   1.0,
   0.9999999999999998,
   0.9999996656343196,
   1.0,
   1.0000000000000002,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.9999999999999999,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.9999999999999999,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.9999999999999999,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.9999999999999999,
   0.9999999999999999,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0000000000000002,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.9999999999999999,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0000000000000002,
   1.0,
   1.0,
   0.9999999999999999,
   1.0,


In [41]:
result, model_outputs, wrong_predictions = model.eval_model(test_df)

Training loss,0.0
lr,0.0
global_step,8980
_runtime,5616
_timestamp,1621017189
_step,328
tp,3459
tn,3725
fp,0
fn,0
mcc,1.0


Training loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,▂▅████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
tp,▁▇██████████████████████████████████████
tn,█▅███▁███████▅▅▅▅▅▅█████████████████████
fp,▁▅▁▁▁█▁▁▁▁▁▁▁▅▅▅▅▅▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
fn,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mcc,▁▇███▇██████████████████████████████████


In [42]:
predictions, raw_outputs = model.predict(test_df['text'])

In [43]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_df['labels'],predictions))

0.5080178173719376


In [ ]:
import pickle
with open(project_path+'Predictions/pickle_Pred_Roberta_1_FR.pickle', 'wb') as f:
  pickle.dump((test_df['labels'],predictions),f)


FileNotFoundError: ignored

In [ ]:
import pickle
with open(project_path+'Predictions/pickle_Pred_Roberta_1_FR.pickle', 'rb') as f:
  y_true, y_pred = pickle.load(f)

FileNotFoundError: ignored

In [39]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))

NameError: ignored